In [45]:
import warnings
warnings.filterwarnings('ignore') # disable warnings relateds to versions of tf
import numpy as np
# keras model and preprocessing tools
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras import backend as K
from keras import utils
# dianna library for explanation
import dianna
from dianna import visualization
# for plotting
%matplotlib inline
from matplotlib import pyplot as plt

2022-09-08 16:25:07.204453: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-08 16:25:07.588065: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-08 16:25:07.588139: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-08 16:25:07.657559: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-08 16:25:08.586768: W tensorflow/stream_executor/platform/de

#### 1 - Loading the model and the dataset
Loads pretrained ImageNet model and the image to be explained.

Initialize the pretrained model.

In [36]:
# load the model
import pickle
from pathlib import Path

output_folder = Path('/your_path_to/observational/output')
pkl_model_file_name = str(output_folder / 'LogisticRegressionCV_lag_1_split_0.sav')
loaded_model = pickle.load(open(pkl_model_file_name, 'rb'))                 

In [39]:
# load test data
import pandas as pd
X_test_file_name = output_folder / 'LogisticRegressionCV_X_test_lag_1_split_0.csv'
X_test = pd.read_csv(X_test_file_name, index_col=0)
X_test

,1..1..sst,1..2..sst,1..3..sst,1..4..sst,1..5..sst,1..6..sst,1..7..sst,1..8..sst,1..9..sst,1..11..sst,1..12..sst
1981-01-16,-0.781289,0.026348,-0.154820,0.398731,0.217529,-0.324348,-0.197452,0.262071,0.140344,0.012956,0.006317
1981-02-13,-0.805915,-0.042760,-0.138640,0.388029,0.218031,-0.454347,-0.198092,0.469227,0.005216,-0.014150,-0.022728
1981-03-13,-0.455521,0.062720,-0.190565,0.256821,0.282002,-0.301338,0.080340,0.973964,0.072171,0.055092,0.047149
1981-04-10,-0.359189,-0.090819,-0.097247,-0.001068,0.264594,0.154570,-0.291145,0.091206,0.165278,0.014397,0.045278
1981-05-08,-0.184985,0.086693,0.043705,0.020359,0.546930,-0.133687,0.027764,0.213893,0.207878,0.106212,0.058237
...,...,...,...,...,...,...,...,...,...,...,...
1991-08-28,0.225965,-0.105548,-0.328257,-0.479323,0.279319,-0.469070,0.025348,-0.035842,-0.076151,-0.115672,-0.249279
1991-09-25,0.427475,-0.150174,-0.471862,-0.287656,0.189109,-0.311486,0.113529,-0.202739,-0.044259,-0.153844,-0.170419
1991-10-23,0.659344,-0.115526,-0.624655,0.002071,0.328530,-0.227904,-0.099115,-0.333195,-0.142204,-0.365160,-0.225685
1991-11-20,0.797536,-0.024138,-0.501468,-0.142064,0.275873,-0.071305,-0.059220,-0.164603,-0.110576,-0.410891,-0.550052


In [48]:
class Model():
    def __init__(self, input_size):
        K.set_learning_phase(0)
        self.model = loaded_model
        self.input_size = input_size
        
    def run_on_batch(self, x):
        return self.model.predict_proba(x)

In [49]:
model = Model(X_test.shape)

Load and preprocess image.

#### 2 - Compute and visualize the relevance scores
Compute the pixel relevance scores using RISE and visualize them on the input image. 

RISE masks random portions of the input image and passes the masked image through the model — the masked portion that decreases accuracy the most is the most “important” portion.<br>
To call the explainer and generate relevance scores map, the user need to specifiy the number of masks being randomly generated (`n_masks`), the resolution of features in masks (`feature_res`) and for each mask and each feature in the image, the probability of being kept unmasked (`p_keep`).

In [ ]:
# TODO the shape of X_test should be changed from (143, 11) to (1, 143, 11)
relevances = dianna.explain_image(model.run_on_batch, X_test, method="RISE",
                                labels=[i for i in range(1)],
                                n_masks=1000, feature_res=6, p_keep=.1,
                                axis_labels={0: 'channels'})

Make predictions and select the top prediction.


In [ ]:
def class_name(idx):
    return decode_predictions(np.eye(1, 1000, idx))[0][0][1]

# print the name of predicted class, taking care of adding a batch axis to the model input
class_name(np.argmax(model.model.predict(x[None, ...])))

Visualize the relevance scores for the predicted class on top of the input image.

In [ ]:
class_idx = np.argmax(model.model.predict(x[None, ...]))
print(f'Explanation for `{class_name(class_idx)}`')
visualization.plot_image(relevances[class_idx], utils.img_to_array(img)/255., heatmap_cmap='jet')

#### 3 - Conclusions
The relevance scores are generated by passing multiple randomly masked inputs to the black-box model and averaging their scores. The idea behind this is that whenever a mask preserves important parts of the image it gets higher score. <br>

The example here shows that the RISE method evaluates the relevance of each pixel/super pixel to the classification. Pixels characterizing the bee are highlighted by the XAI approach, which gives an intuition on how the model classifies the image. The results are reasonable, based on the human visual preception of the image.